In [1]:
from progressivis_nb_widgets import *

In [2]:
from progressivis import Scheduler, Every#, log_level
from progressivis.cluster import MBKMeans, MBKMeansFilter
from progressivis.io import CSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
from progressivis.stats import RandomTable
from progressivis.core.utils import gather_and_run
from progressivis.utils.psdict import PsDict
import pandas as pd
import numpy as np
import os.path
import tempfile
from progressivis.datasets.random import generate_random_multivariate_normal_csv as gen_csv
try:
    s = scheduler
except NameError:
    s = Scheduler()
    #log_level(package="progressivis.cluster")

dir_name = os.path.join(tempfile.gettempdir(), 'progressivis_tmp_')
os.makedirs(dir_name, exist_ok=True)
file_name = os.path.join(dir_name, "foobar.csv")
gen_csv(file_name, rows=1_000_000, reset=True) 
data = CSVLoader(file_name, skipinitialspace=True, header=None, index_col=False,scheduler=s)
n_clusters = 3
mbkmeans = MBKMeans(columns=['_0', '_1'], n_clusters=n_clusters, 
                    batch_size=100, tol=0.01, is_input=False, scheduler=s)
mbkmeans.create_dependent_modules(data)
classes = []
for i in range(n_clusters):
    cname = f"k{i}"
    filt = MBKMeansFilter(i, scheduler=s)
    filt.create_dependent_modules(mbkmeans, data, 'result')
    classes.append({'name': cname, 'x_column': '_0',
                    'y_column': '_1', 'sample': mbkmeans if i==0 else None,
                    'sample_slot': 'result',
                    'input_module': filt, 'input_slot': 'result'})
sp = MCScatterPlot(scheduler=s, classes=classes)
sp.create_dependent_modules()
for i in range(n_clusters):
    cname = f"k{i}"
    sp[cname].min_value.result = PsDict({'_0': -np.inf, '_1': -np.inf})
    sp[cname].max_value.result = PsDict({'_0': np.inf, '_1': np.inf})

sp.move_point = mbkmeans.moved_center # for input management
# Create the dashboard object
psboard = PsBoard(s, order='desc')
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(disable=['filter']), sp)
# 2) When widget is generic or for overloading the link_module() method do:
# psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :
gather_and_run(s.start(), *psboard.coroutines)

#### Click  _&#x1f441; mc_scatter_plot_1_ below to display the scatterplot:

In [3]:
display(psboard)

PsBoard(children=(ControlPanel(children=(Button(description='Resume', disabled=True, icon='play', style=Button…